In [2]:
import sys
sys.path.append('../lib/')

import pandas as pd
import numpy as np
import geopandas as gpd

import math

import scipy.interpolate
from copy import deepcopy
from shapely.geometry import Point
from collections import defaultdict

In [4]:
grids = gpd.read_file('../results/grids_sweden/grids_1km_density_deso_with_10km_upper_grids.shp')

In [5]:
#Filter out the grids which do not have the density data
deleteIndex = []

within_1 = defaultdict(list)


for index, row in grids.iterrows():
    if row['deso'] == None or math.isnan(row['density']):
        deleteIndex.append(index)
    else:
        within_1[row['deso']].append(row['zone'])
grids = grids.drop(index = deleteIndex)

little_name = list(grids.zone)
population_density = dict(zip(grids.zone, grids.density))
geometry = dict(zip(grids.zone, grids.geometry))


In [6]:
grids.head()

,zone,deso,density,upper_deso,upper_dens,upper_zone,upper_xcoo,upper_ycoo,geometry
0,18,0114A0010,15.720316,0114A0010,15.720316,1,655000.0,6595000.0,"POLYGON ((660000.000 6596000.000, 660000.000 6..."
1,8,0114A0010,15.720316,0114A0010,15.720316,1,655000.0,6595000.0,"POLYGON ((659000.000 6598000.000, 659000.000 6..."
2,20,0114A0010,15.720316,0114A0010,15.720316,1,655000.0,6595000.0,"POLYGON ((660000.000 6598000.000, 660000.000 6..."
3,19,0114A0010,15.720316,0114A0010,15.720316,1,655000.0,6595000.0,"POLYGON ((660000.000 6597000.000, 660000.000 6..."
4,1,0114A0010,15.720316,0114A0010,15.720316,1,655000.0,6595000.0,"POLYGON ((658000.000 6600000.000, 658000.000 6..."


In [7]:
big_grid = grids[["upper_dens", "upper_zone", "upper_xcoo", "upper_ycoo"]].drop_duplicates()

In [8]:
name_big = list(big_grid.upper_zone)
den_big = dict(zip(big_grid.upper_zone, big_grid.upper_dens))
geo_big = dict(zip(big_grid.upper_zone, list(zip(big_grid.upper_xcoo, big_grid.upper_ycoo))))

In [9]:
base_info = gpd.read_file('../dbs/sweden/zones/DeSO/DeSO_2018_v2.shp')
#base_info.loc[:, 'deso_3'] = base_info.loc[:, 'deso'].apply(lambda x: x[:2])
#zones_subset = base_info.loc[base_info['deso_3'] == '14', :]
#zones_subset_info = dict(zip(zones_subset['deso'], zones_subset['geometry']))
#zone_name = list(zones_subset['deso'])
zones_info = dict(zip(base_info['deso'], base_info['geometry']))
zone_name = list(base_info['deso'])

In [10]:
cover = []
checked = set()
for i in range(0, len(zone_name)):
    sub_cover = []
    for j in range(0, len(name_big)):
        if j in checked:
            continue
        point_j = Point(geo_big[name_big[j]][0], geo_big[name_big[j]][1])
        if zones_info[zone_name[i]].contains(point_j) == True:
            # grid_j in zone_i
            sub_cover.append(name_big[j])
            checked.add(j) # This grid has been occupied, we do not need to check it again.
    cover.append(sub_cover)
within = dict(zip(zone_name, cover))


bigzone_name = []
bigCover = []
subCover = []
old_name = zone_name[0][0:5]

bigCover_1 = []
subCover_1 = []

bigzone_name.append(old_name)
subCover.extend(within[zone_name[0]])
subCover_1.extend(within_1[zone_name[0]])

for i in range(1, len(zone_name)):
    new_name = zone_name[i][0:5]

    if new_name == old_name:
        # this two zones belong the same big zone
        subCover.extend(within[zone_name[i]])
        subCover_1.extend(within_1[zone_name[i]])

    if new_name != old_name:
        # find a new big zone
        #store old results
        bigCover.append(deepcopy(subCover))
        subCover.clear()

        bigCover_1.append(deepcopy(subCover_1))
        subCover_1.clear()

        #store new resutls
        bigzone_name.append(new_name)
        subCover.extend(within[zone_name[i]])
        subCover_1.extend(within_1[zone_name[i]])

    old_name = new_name

# handle the lastest case
bigCover.append(subCover)
bigCover_1.append(subCover_1)

big_within = dict(zip(bigzone_name, bigCover))
big_within_1 = dict(zip(bigzone_name, bigCover_1))

In [11]:
print(bigzone_name)

['0114A', '0114C', '0115A', '0115B', '0115C', '0117A', '0117C', '0120A', '0120C', '0123C', '0125A', '0125B', '0125C', '0126A', '0126C', '0127A', '0127C', '0128A', '0128C', '0136A', '0136B', '0136C', '0138A', '0138C', '0139A', '0139B', '0139C', '0140A', '0140C', '0160C', '0162C', '0163C', '0180C', '0181A', '0181B', '0181C', '0182C', '0183C', '0184C', '0186C', '0187A', '0187C', '0188A', '0188B', '0188C', '0191A', '0191B', '0191C', '0192A', '0192B', '0192C', '0305A', '0305B', '0305C', '0319A', '0319B', '0319C', '0330A', '0330C', '0331A', '0331B', '0331C', '0360A', '0360B', '0360C', '0380A', '0380B', '0380C', '0381A', '0381B', '0381C', '0382A', '0382B', '0382C', '0428A', '0428C', '0461A', '0461C', '0480A', '0480B', '0480C', '0481C', '0482A', '0482B', '0482C', '0483A', '0483B', '0483C', '0484A', '0484B', '0484C', '0486A', '0486B', '0486C', '0488A', '0488B', '0488C', '0509A', '0509C', '0512A', '0512C', '0513A', '0513B', '0513C', '0560A', '0560C', '0561A', '0561B', '0561C', '0562A', '0562C', 

In [9]:
df_data = pd.read_csv('../results/distance_ratio_data.csv')
df_simulation = pd.read_csv('../results/distance_ratio_simulation.csv')


sp_data_sweden = scipy.interpolate.interp1d(df_data.loc[df_data.country == 'sweden', ['distance']].values.reshape(-1),
                                            df_data.loc[df_data.country == 'sweden', ['ratio']].values.reshape(-1), bounds_error = False, fill_value = 1.5)

sp_data_simulation = scipy.interpolate.interp1d(df_simulation.loc[:, ['distance']].values.reshape(-1),
                                                df_simulation.loc[:, ['ratio']].values.reshape(-1),  bounds_error = False, fill_value = 1.5)

In [10]:
model_output = dict()
demand_d = dict()
demand_D_survey = dict()
demand_D_simulation = dict()


area_1 = 1
r_average_1 = area_1 / math.pi

area_5 = 100
r_average_5= area_5 / math.pi

T = 1000
f_max = 1
f_min = 1/T
parameter = math.log(f_max / f_min)

p_1 = area_1 * r_average_1 * parameter
p_5 = area_5 * r_average_5 * parameter


for i in range(0, len(bigzone_name)):
    element = dict()
    element1 = dict()
    element2 = dict()
    element3 = dict()
    for j in range(i, len(bigzone_name)): 
        average_daily_trips = 0
        demand_d_tmp = 0
        demand_D_tmp = 0
        demand_D_simulation_tmp = 0
        if i == j:
            for begin in range(0, len(big_within_1[bigzone_name[i]])):
                for end in range(0, len(big_within_1[bigzone_name[j]])):
                    if begin != end:
                        deltax = (geometry[big_within_1[bigzone_name[i]][begin]].centroid.x - geometry[big_within_1[bigzone_name[j]][end]].centroid.x) / 1000                    
                        deltay = (geometry[big_within_1[bigzone_name[i]][begin]].centroid.y - geometry[big_within_1[bigzone_name[j]][end]].centroid.y) / 1000
                
                        distance = deltax * deltax + deltay * deltay

                        root_distance = math.sqrt(distance)
                    
                        tmp =  ( population_density[big_within_1[bigzone_name[i]][begin]] + population_density[big_within_1[bigzone_name[j]][end]] ) / distance
                        tmp1 = tmp * root_distance
                        tmp2 = tmp * float(sp_data_sweden(root_distance)) * root_distance
                        tmp3 = tmp * float(sp_data_simulation(root_distance)) * root_distance

                        average_daily_trips = average_daily_trips + tmp
                        demand_d_tmp = demand_d_tmp + tmp1
                        demand_D_tmp = demand_D_tmp + tmp2
                        demand_D_simulation_tmp = demand_D_simulation_tmp + tmp3

            element[bigzone_name[j]] = p_1 * average_daily_trips
            element1[bigzone_name[j]] = p_1 * demand_d_tmp
            element2[bigzone_name[j]] = p_1 * demand_D_tmp
            element3[bigzone_name[j]] = p_1 * demand_D_simulation_tmp
        else:
            for begin in range(0, len(big_within[bigzone_name[i]])):
                for end in range(0, len(big_within[bigzone_name[j]])):
                        deltax = (geo_big[big_within[bigzone_name[i]][begin]][0] - geo_big[big_within[bigzone_name[j]][end]][0]) / 1000                    
                        deltay = (geo_big[big_within[bigzone_name[i]][begin]][1] - geo_big[big_within[bigzone_name[j]][end]][1]) / 1000
                
                        distance = deltax * deltax + deltay * deltay

                        root_distance = math.sqrt(distance)
                    
                        tmp =  ( den_big[big_within[bigzone_name[i]][begin]] + den_big[big_within[bigzone_name[j]][end]] ) / distance
                        tmp1 = tmp * root_distance
                        tmp2 = tmp * float(sp_data_sweden(root_distance)) * root_distance
                        tmp3 = tmp * float(sp_data_simulation(root_distance)) * root_distance

                        average_daily_trips = average_daily_trips + tmp
                        demand_d_tmp = demand_d_tmp + tmp1
                        demand_D_tmp = demand_D_tmp + tmp2
                        demand_D_simulation_tmp = demand_D_simulation_tmp + tmp3

            element[bigzone_name[j]] = p_5 * average_daily_trips
            element1[bigzone_name[j]] = p_5 * demand_d_tmp
            element2[bigzone_name[j]] = p_5 * demand_D_tmp
            element3[bigzone_name[j]] = p_5 * demand_D_simulation_tmp
    model_output[bigzone_name[i]] = element
    demand_d[bigzone_name[i]] = element1
    demand_D_survey[bigzone_name[i]] = element2
    demand_D_simulation[bigzone_name[i]] = element3

In [11]:
results_output = "../results/model_output_10km_1km_sweden.txt"
results_demand_d = '../results/demand_d_10km_1km_sweden.txt'
results_demand_D_servey = '../results/demand_D_survey_10km_1km_sweden.txt'
results_demand_D_simulation = '../results/demand_D_simulation_10km_1km_sweden.txt'

In [12]:
def store(results_output, model_output):
    result = np.zeros(len(bigzone_name)*len(bigzone_name))
    for i in range(0, len(bigzone_name)):
        for j in range(0, len(bigzone_name)):
            if i <= j:
                result[i + j * len(bigzone_name)] = model_output[bigzone_name[i]][bigzone_name[j]]
            if i > j:
                result[i + j * len(bigzone_name)] = model_output[bigzone_name[j]][bigzone_name[i]]
    np.savetxt(results_output, result)

In [13]:
store(results_output, model_output)
store(results_demand_d, demand_d)
store(results_demand_D_servey, demand_D_survey)
store(results_demand_D_simulation, demand_D_simulation)